# Census Data API: ACS 2019 Statewide Population 

In [34]:
from urllib import request
import json

census_api_key= '75b228a93b0b0592dd6d820dc0077020a546514a'
url_str= 'https://api.census.gov/data/2019/acs/acs5?get=B01003_001E,NAME,GEO_ID&for=state:*&key='+census_api_key

predicates= {}
get_vars= ["NAME","GEO_ID","B01003_001E"]
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:*"
r= requests.get(url_str, params= predicates)

In [35]:
print(r.text)

[["B01003_001E","NAME","GEO_ID","state"],
["4876250","Alabama","0400000US01","01"],
["737068","Alaska","0400000US02","02"],
["7050299","Arizona","0400000US04","04"],
["2999370","Arkansas","0400000US05","05"],
["39283497","California","0400000US06","06"],
["5610349","Colorado","0400000US08","08"],
["957248","Delaware","0400000US10","10"],
["692683","District of Columbia","0400000US11","11"],
["3575074","Connecticut","0400000US09","09"],
["20901636","Florida","0400000US12","12"],
["10403847","Georgia","0400000US13","13"],
["1717750","Idaho","0400000US16","16"],
["1422094","Hawaii","0400000US15","15"],
["12770631","Illinois","0400000US17","17"],
["6665703","Indiana","0400000US18","18"],
["3139508","Iowa","0400000US19","19"],
["2910652","Kansas","0400000US20","20"],
["4449052","Kentucky","0400000US21","21"],
["4664362","Louisiana","0400000US22","22"],
["1335492","Maine","0400000US23","23"],
["6018848","Maryland","0400000US24","24"],
["6850553","Massachusetts","0400000US25","25"],
["9965265

In [36]:
print(r.json()[0])

['B01003_001E', 'NAME', 'GEO_ID', 'state']


In [37]:
col_names= ["total_pop","Name", "GEO_ID", "state"]

In [38]:
import pandas as pd
df= pd.DataFrame (columns=col_names, data=r.json()[1:])
#fix data types
df["total_pop"]= df["total_pop"].astype(int)
df.head()

,total_pop,Name,GEO_ID,state
0,4876250,Alabama,0400000US01,01
1,737068,Alaska,0400000US02,02
2,7050299,Arizona,0400000US04,04
3,2999370,Arkansas,0400000US05,05
4,39283497,California,0400000US06,06


In [39]:
# df.to_csv('TEST.csv', encoding='utf-8', index=False)

# Census Data TARGET: Mapping

In [45]:
import geopandas as gpd
import requests
import zipfile
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt

%matplotlib inline 
# jupyter "magic" to display plots in notebook

In [46]:
url_2 ="https://www2.census.gov/geo/tiger/TIGER2020/STATE/tl_2020_us_state.zip"

In [47]:
local_path = 'tmp/'

print('Downloading shapefile...')

r = requests.get(url_2)
z = zipfile.ZipFile(io.BytesIO(r.content))
print("Done")

z.extractall(path=local_path) 
# extract to folder

filenames = [y for y in sorted(z.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
print(filenames)

Done
['tl_2020_us_state.dbf', 'tl_2020_us_state.prj', 'tl_2020_us_state.shp', 'tl_2020_us_state.shx']


In [48]:
dbf, prj, shp, shx = [filename for filename in filenames]
usa = gpd.read_file(local_path + shp)
print("Shape of the dataframe: {}".format(usa.shape))
print("Projection of dataframe: {}".format(usa.crs))

usa.tail() 
#last 5 records in dataframe

usa.head()

Shape of the dataframe: (56, 15)
Projection of dataframe: {'init': 'epsg:4269'}


,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62266296765,489206049,+38.6472854,-080.6183274,"POLYGON ((-81.74725 39.09538, -81.74635 39.096..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138958484319,45975808217,+28.3989775,-082.5143005,"MULTIPOLYGON (((-86.39964 30.22696, -86.40262 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143778461053,6216594318,+40.1028754,-089.1526108,"POLYGON ((-91.18529 40.63780, -91.17510 40.643..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206232157570,18949864226,+46.3159573,-094.1996043,"POLYGON ((-96.78438 46.63050, -96.78434 46.630..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25151895765,6979171386,+38.9466584,-076.6744939,"POLYGON ((-77.45881 39.22027, -77.45866 39.220..."


In [49]:
df2= pd.DataFrame (usa, columns= ['GEOID', 'NAME', 'geometry'])
print (df)

    total_pop                  Name       GEO_ID state
0     4876250               Alabama  0400000US01    01
1      737068                Alaska  0400000US02    02
2     7050299               Arizona  0400000US04    04
3     2999370              Arkansas  0400000US05    05
4    39283497            California  0400000US06    06
5     5610349              Colorado  0400000US08    08
6      957248              Delaware  0400000US10    10
7      692683  District of Columbia  0400000US11    11
8     3575074           Connecticut  0400000US09    09
9    20901636               Florida  0400000US12    12
10   10403847               Georgia  0400000US13    13
11    1717750                 Idaho  0400000US16    16
12    1422094                Hawaii  0400000US15    15
13   12770631              Illinois  0400000US17    17
14    6665703               Indiana  0400000US18    18
15    3139508                  Iowa  0400000US19    19
16    2910652                Kansas  0400000US20    20
17    4449

In [50]:
print (type('GEOID'))

<class 'str'>


In [52]:
df2['Geo_ID']= '0400000US' + df2['GEOID'].astype(str)
print (df2)

   GEOID                                          NAME  \
0     54                                 West Virginia   
1     12                                       Florida   
2     17                                      Illinois   
3     27                                     Minnesota   
4     24                                      Maryland   
5     44                                  Rhode Island   
6     16                                         Idaho   
7     33                                 New Hampshire   
8     37                                North Carolina   
9     50                                       Vermont   
10    09                                   Connecticut   
11    10                                      Delaware   
12    35                                    New Mexico   
13    06                                    California   
14    34                                    New Jersey   
15    55                                     Wisconsin   
16    41      

In [66]:
df3= df2.set_index('Geo_ID').join(df.set_index('GEO_ID'))
print (df3)

            GEOID                                          NAME  \
Geo_ID                                                            
0400000US54    54                                 West Virginia   
0400000US12    12                                       Florida   
0400000US17    17                                      Illinois   
0400000US27    27                                     Minnesota   
0400000US24    24                                      Maryland   
0400000US44    44                                  Rhode Island   
0400000US16    16                                         Idaho   
0400000US33    33                                 New Hampshire   
0400000US37    37                                North Carolina   
0400000US50    50                                       Vermont   
0400000US09    09                                   Connecticut   
0400000US10    10                                      Delaware   
0400000US35    35                                    New Mexic

In [67]:
df3.to_csv('Pop_State.csv', index=True)

In [68]:
df3= gpd.GeoDataFrame(df3, geometry='geometry')

In [69]:
df3.to_file('MyGeometries.shp', driver='ESRI Shapefile')